# Introduction

- <span style="color:lightgreen">***Generic**: Every programming language has tools for effectively handling the duplication of concepts. In Rust, one such tool is **generics: abstract stand-ins for concrete types or other properties**. Functions can take parameters of some generic type, e.g. `Option<T>`, `Vec<T>`..., instead of a concrete type like `i32` or `String`*</span>

- <span style="color:lightgreen">***Traits: We can use traits to define behavior in a generic way**. You can combine traits with generic types to constrain a generic type to accept only those types that have a particular behavior, as opposed to just any type.*</span>

- <span style="color:lightgreen">***Lifetimes: a variety of generics that give the compiler information about how references relate to each other.***. *Lifetimes allow us to give the compiler enough information about borrowed values so that it can ensure references will be valid in more situations than it could without our help.*</span>


# Generic Data Types

<span style="color:skyblue">***We use generics to create definitions for items like function signatures or structs, which we can then use with many different concrete data types.***</span>

## In Function Definitions

Instead of writing 2 functions that differ only in the type they operate on:
- `fn largest_i32(list: &[i32]) -> &i32`, and
- `fn largest_char(list: &[char]) -> &char`
We can define a function that takes in a generic type `T` with 
```rust
fn largest<T>(list: &[T]) -> &T {
    // snip
}
```
like below


In [2]:
fn largest<T>(list: &[T]) -> &T {
    let mut largest = &list[0];

    for item in list {
        if item > largest {
            largest = item;
        }
    }

    largest
}

fn main() {
    let number_list = vec![34, 50, 25, 100, 65];

    let result = largest(&number_list);
    println!("The largest number is {}", result);

    let char_list = vec!['y', 'm', 'a', 'q'];

    let result = largest(&char_list);
    println!("The largest char is {}", result);
}

Error: binary operation `>` cannot be applied to type `&T`

<span style="color:orange">***We got an error that mentions the `std::cmp::PartialOrd` traits, and this error states that the body of `largest` won’t work for all possible types that `T` could be, but only those that implement the `PartialOrd` trait.***</span>

## In Struct Definitions

Below we defines `Point<T>` to hold values of both `int` and `float`, but `x` and `y` must be of the same type:

In [8]:
#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

fn main() {
    let integer = Point { x: 5, y: 10 };
    println!("integer point = {:?}", integer);
    let float = Point { x: 1.0, y: 4.0 };
    println!("float point = {:?}", float);
}

main()

integer point = Point { x: 5, y: 10 }
float point = Point { x: 1.0, y: 4.0 }


()

We can make a `Point<T, U>` struct where `x` and `y` have different types like below

In [11]:
#[derive(Debug)]
struct Point<T, U> {
    x: T,
    y: U,
}

fn main() {
    let both_integer = Point { x: 5, y: 10 };
    println!("both int point = {:?}", both_integer);
    let both_float = Point { x: 1.0, y: 4.0 };
    println!("both float point = {:?}", both_float);
    let integer_and_float = Point { x: 5, y: 4.0 };
    println!("int & float point = {:?}", integer_and_float);
}

main()

both int point = Point { x: 5, y: 10 }
both float point = Point { x: 1.0, y: 4.0 }
int & float point = Point { x: 5, y: 4.0 }


()

## In Enum Definitions

Similar to structs, we can define enums with generic types, e.g. the `Option<T>`
```rust
enum Option<T> {
    Some(T),
    None,
}
```
and the enum `Result<T, E>`
```rust
Result<T, E> {
    Ok(T),
    Err(E),
}
```
that has 2 two variants: `Ok`, which holds a value of type `T`, and `Err`, which holds a value of type `E`.

## In Method Definitions

<span style="color:skyblue">***We can implement methods on structs and enums and use generic types in their definitions, too.***</span>

In [20]:
struct Point<T> {
    x: T,
    y: T,
}

impl<T> Point<T> {
    fn x(&self) -> &T {
        &self.x
    }
}

// We can also specify constraints on generic types when defining methods on the type
// This function only works on Point<f32>
impl Point<f32> {
    fn distance_from_origin(&self) -> f32 {
        (self.x.powi(2) + self.y.powi(2)).sqrt()
    }
}

fn main() {
    let p1 = Point { x: 5, y: 10 };
    println!("p1.x = {}", p1.x());
    let p2 = Point { x: 1.1, y: 10.3 };
    println!("p2.x = {}", p2.x());


    println!("distance from origin of p2 = {}", p2.distance_from_origin());

    // this will throw error since `distance_from_origin` is not implemented for `Point<{integer}>`
    // println!("distance from origin of p1 = {}", p1.distance_from_origin());
}

main()

p1.x = 5
p2.x = 1.1
distance from origin of p2 = 10.358571


()

Generic type parameters in a struct definition aren’t always the same as those you use in that same struct’s method signatures.

In [23]:
struct Point<X1, Y1> {
    x: X1,
    y: Y1,
}

// Some generic parameters are declared with impl and some are declared with the method definition. 
// Here, the generic parameters X1 and Y1 are declared after impl because they go with 
// the struct definition. The generic parameters X2 and Y2 are declared after fn mixup, 
// because they’re only relevant to the method.
impl<X1, Y1> Point<X1, Y1> {
    fn mixup<X2, Y2>(self, other: Point<X2, Y2>) -> Point<X1, Y2> {
        Point {
            x: self.x,
            y: other.y,
        }
    }
}

fn main() {
    let p1 = Point { x: 5, y: 10.4 };
    let p2 = Point { x: "Hello", y: 'c' };
    println!("p1.x = {}, p1.y = {}", p1.x, p1.y);
    println!("p2.x = {}, p2.y = {}", p2.x, p2.y);
    
    let p3 = p1.mixup(p2);

    println!("p3.x = {}, p3.y = {}", p3.x, p3.y);
}

main()

p1.x = 5, p1.y = 10.4
p2.x = Hello, p2.y = c
p3.x = 5, p3.y = c


()

## Performance of Code Using Generics

<span style="color:skyblue">*Using generic types won't make your program run any slower than it would with concrete types, since Rust performs monomorphization of the code using generics at compile time. **Monomorphization is the process of turning generic code into specific code by filling in the concrete types that are used when compiled**.*</span>

# Traits: Defining Shared Behavior

<span style="color:lightgreen">***A trait (similar to interfaces in some languages) defines functionality a particular type has and can share with other types**. We can use traits to define shared behavior in an abstract way. We can use trait bounds to specify that a generic type can be any type that has certain behavior.*</span>

## Defining a Trait

A type’s behavior consists of the methods we can call on that type. Different types share the same behavior if we can call the same methods on all of those types. Trait definitions are a way to group method signatures together to define a set of behaviors necessary to accomplish some purpose.

For example, let say we have 2 structs `NewsArticle` and `Tweet` that hold texts, both need to have a `summarize` method on the instance of each type to display the summarization => these 2 structs shares the `summarize` behavior, or the `Summary` trait with the `summarize` method defined like below

In [24]:
pub trait Summary {
    fn summarize(&self) -> String;
}

## Implementing a Trait on a Type

Now that we’ve defined the desired signatures of the `Summary` trait’s methods, we can implement it on the `NewsArticle` and `Tweet` types

In [29]:
pub trait Summary {
    fn summarize(&self) -> String;
}

pub struct NewsArticle {
    pub headline: String,
    pub location: String,
    pub author: String,
    pub content: String,
}

impl Summary for NewsArticle {
    fn summarize(&self) -> String {
        format!("{}, by {} from {}", self.headline, self.author, self.location)
    }
}

pub struct Tweet {
    pub username: String,
    pub content: String,
    pub reply: bool,
    pub retweet: bool,
}

impl Summary for Tweet {
    fn summarize(&self) -> String {
        format!("{}: {}", self.username, self.content)
    }
}


fn main() {
    let tweet = Tweet {
        username: String::from("horse_ebooks"),
        content: String::from(
            "of course, as you probably already know, people",
        ),
        reply: false,
        retweet: false,
    };

    println!("1 new tweet: {}", tweet.summarize());


    let news = NewsArticle {
        headline: String::from("Hello World!"),
        location: String::from("anywhere"),
        author: String::from("anyone"),
        content: String::from("say hello to the world!"),
    };

    println!("1 new news article: {}", news.summarize());
}

main()


1 new tweet: horse_ebooks: of course, as you probably already know, people
1 new news article: Hello World!, by anyone from anywhere


()

> 📓 **Note**: *Let's say we make a crate called `aggregator` that holds the `Summary` trait*
> - *We can implement a trait on a type only if at least one of the trait or the type is local to our crate, e.g. we can implement the `Display` trait on our `Tweet` type*
> - *But we can’t implement external traits on external types, e.g. we can’t implement the `Display` trait on `Vec<T>` within our `aggregator` crate, because `Display` and `Vec<T>` are both defined in the standard library and aren’t local to our aggregator crate*

## Default Implementations

Sometimes it’s useful to have default behavior for some or all of the methods in a trait instead of requiring implementations for all methods on every type. Then, as we implement the trait on a particular type, we can keep or override each method’s default behavior.

In [5]:
fn main() {

    pub trait Summary {
        fn summarize(&self) -> String {
            String::from("(Read more...)")
        }
    }

    pub struct NewsArticle {
        pub headline: String,
        pub location: String,
        pub author: String,
        pub content: String,
    }

    impl Summary for NewsArticle {}

    let article = NewsArticle {
            headline: String::from("Penguins win the Stanley Cup Championship!"),
            location: String::from("Pittsburgh, PA, USA"),
            author: String::from("Iceburgh"),
            content: String::from(
                "The Pittsburgh Penguins once again are the best \
                hockey team in the NHL.",
            ),
        };

    println!("New article available! {}", article.summarize());
}

main()

New article available! (Read more...)


()

Default implementations can call other methods in the same trait, even if those other methods don’t have a default implementation. In this way, a trait can provide a lot of useful functionality and only require implementors to specify a small part of it.

In [6]:
fn main() {

    pub trait Summary {
        fn summarize_author(&self) -> String;

        fn summarize(&self) -> String {
            format!("(Read more from {}...)", self.summarize_author())
        }
    }

    pub struct Tweet {
        pub username: String,
        pub content: String,
        pub reply: bool,
        pub retweet: bool,
    }

    impl Summary for Tweet {
        fn summarize_author(&self) -> String {
            format!("@{}", self.username)
        }
    }

    let tweet = Tweet {
            username: String::from("horse_ebooks"),
            content: String::from(
                "of course, as you probably already know, people",
            ),
            reply: false,
            retweet: false,
        };

    println!("1 new tweet: {}", tweet.summarize());
}

main()

1 new tweet: (Read more from @horse_ebooks...)


()

## Traits as Parameters

<span style="color:lightgreen">***Let's explore how to use traits to define functions that accept many different types (as long as they implement the specified trait)***</span>. For example, we will define a `notify` function that calls the `summarize` method on its `item` parameter, where `item` can be any type that implements the `Summary` trait.

In [ ]:
fn main() {
    pub trait Summary {
        fn summarize(&self) -> String;
    }

    pub struct NewsArticle {
        pub headline: String,
        pub location: String,
        pub author: String,
        pub content: String,
    }

    impl Summary for NewsArticle {
        fn summarize(&self) -> String {
            format!("{}, by {} ({})", self.headline, self.author, self.location)
        }
    }

    pub struct Tweet {
        pub username: String,
        pub content: String,
        pub reply: bool,
        pub retweet: bool,
    }

    impl Summary for Tweet {
        fn summarize(&self) -> String {
            format!("{}: {}", self.username, self.content)
        }
    }

    pub fn notify(item: &impl Summary) {
        println!("Breaking news! {}", item.summarize());
    }
}

### Trait Bound Syntax

<span style="color:skyblue"> *We can also use this **trait bound syntax** to enforce 2 trait parameters of the same type*</span>
```rust
pub fn notify<T: Summary>(item1: &T, item2: &T) {
```

### Specifying Multiple Trait Bounds with the `+` Syntax

If we want to specify in the `notify` definition that `item` must implement both `Display` and `Summary` traits, we use the following syntax
```rust
pub fn notify(item: &(impl Summary + Display)) {
```
which also works with the trait bounds syntax
```rust
pub fn notify<T: Summary + Display>(item: &T) {
```

### Clearer Trait Bounds with `where` Clauses
Instead of writing
```rust
fn some_function<T: Display + Clone, U: Clone + Debug>(t: &T, u: &U) -> i32 {
```
which is very hard to read, Rust has the `where` syntax for multiple generic type parameters:
```rust
fn some_function<T, U>(t: &T, u: &U) -> i32
where
    T: Display + Clone,
    U: Clone + Debug,
{
```

## Returning Types that Implement Traits

We can also use the `impl Trait` syntax in the return position to return a value of some type that implements a trait:
```rust
fn returns_summarizable() -> impl Summary {
    Tweet {
        username: String::from("horse_ebooks"),
        content: String::from(
            "of course, as you probably already know, people",
        ),
        reply: false,
        retweet: false,
    }
}
```
However, we can only return a single type that implements the trait, so the code below won't work:
```rust
fn returns_summarizable(switch: bool) -> impl Summary {
    if switch {
        NewsArticle {
            // snip
        }
    } else {
        Tweet {
            // snip
        }
    }
}
```

## Using Trait Bounds to Conditionally Implement Methods

By using a trait bound with an `impl` block that uses generic type parameters, we can implement methods conditionally for types that implement the specified traits.

In [ ]:
use std::fmt::Display;

struct Pair<T> {
    x: T,
    y: T,
}

impl<T> Pair<T> {  // always implements the `new` function to return a new instance of `Pair<T>`
    fn new(x: T, y: T) -> Self {
        Self { x, y }
    }
}

impl<T: Display + PartialOrd> Pair<T> {  // Pair<T> only implements the `cmp_display` method 
                                         // if its inner type `T` implements the `PartialOrd` trait 
                                         // that enables comparison and the `Display` trait that enables printing
    fn cmp_display(&self) {
        if self.x >= self.y {
            println!("The largest member is x = {}", self.x);
        } else {
            println!("The largest member is y = {}", self.y);
        }
    }
}

<span style="color:lightgreen">*We can also conditionally implement a trait for any type that implements another trait (i.e. **blanket implementations**)*</span>.  
For example, the standard library implements the `ToString` trait on any type that implements the `Display` trait
```rust
impl<T: Display> ToString for T {
    // --snip--
}
```

In [10]:
let s = 3.to_string();  // since 3 (i32) implements the Display trait, we can call the `to_string` method
s

"3"

<span style="color:yellow">💡*Traits and trait bounds let us write code that uses generic type parameters to reduce duplication but also specify to the compiler that we want the generic type to have particular behavior. The compiler can then use the trait bound information to check that all the concrete types used with our code provide the correct behavior.*</span>

# Lifetimes for Validating References

<span style="color:lightgreen">*Lifetimes are **another kind of generic** that we’ve already been using. Rather than ensuring that a type has the behavior we want, **lifetimes ensure that references are valid as long as we need them to be**. Most of the time, lifetimes are implicit and inferred, and we **must annotate lifetimes when the lifetimes of references could be related in a few different ways***</span>

## Preventing Dangling References with Lifetimes
<span style="color:lightgreen">***The main aim of lifetimes is to prevent dangling references, which cause a program to reference data other than the data it’s intended to reference.***</span>  
Consider the below code which gives the error `x does not live long enough`

In [15]:
fn main() {
    let r;
    {
        let x = 5;
        r = &x;
    }
    println!("r: {}", r);
}

Error: `x` does not live long enough

The reason for the error is that `r` shoud not reference memory that was deallocated when `x` went out of scope

## The Borrow Checker

<span style="color:salmon">***The Rust compiler has a borrow checker that compares scopes to determine whether all borrows are valid***</span>
```rust
fn main() {
    let r;                // ---------+-- 'a
                          //          |
    {                     //          |
        let x = 5;        // -+-- 'b  |
        r = &x;           //  |       |
    }                     // -+       |
                          //          |
    println!("r: {}", r); //          |
}                         // ---------+
```
<span style="color:orange">*Here, we’ve annotated the lifetime of `r` with `'a` and the lifetime of `x` with `'b`. As you can see, the inner `'b` block is much smaller than the outer `'a` lifetime block. At compile time, Rust compares the size of the two lifetimes and sees that `r` has a lifetime of `'a` but that it refers to memory with a lifetime of `'b`. The program is rejected because `'b` is shorter than `'a`: the subject of the reference doesn’t live as long as the reference.*</span>

To fix the above code, we have something like below with a valid reference because the data `x` has a longer lifetime than the reference `r`
```rust
fn main() {
    let x = 5;            // ----------+-- 'b
                          //           |
    let r = &x;           // --+-- 'a  |
                          //   |       |
    println!("r: {}", r); //   |       |
                          // --+       |
}                         // ----------+
```